# Prototype (mostly copy and paste) of LangChain agent code

Please integrate to main script later. Or don't.

### I want to log stuff to make life easier. LangSmith?

LangSmith allows us to "Observe, Evaluate, Develop and Deploy". Is it ClearML tracking for LLMs? Yes. Is it inferior? Of course! Nothing is better than TransparentML!

LangSmith supports searching online or retrieving over local index. This is cool, but I'm gonna ignore Tavily for now.

In [1]:
%pip install -U langchain langgraph langchain-nvidia-ai-endpoints langchain-community langchain-openai tavily-python geocoder

Note: you may need to restart the kernel to use updated packages.


### Setup of env, data, embedder, etc.

Available NVIDIA api keys; (please auto-delete if it no longer works, and help to create a new one)

1. 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
2. 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
3. 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
4. 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

In [49]:
import getpass
import os

nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
os.environ["NVIDIA_API_KEY"] = nvidia_api_key

LangSmith API key(s?):

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

In [5]:
!export LANGCHAIN_TRACING_V2=true
!export LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
!export LANGCHAIN_API_KEY="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"
!export LANGCHAIN_PROJECT="cvu-os-NIMRAG"

In [50]:
# Can choose any model hosted at Nvidia API Catalog (Uncomment the below code to list the availabe models)
# additional caviat; model must support tools?
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
# llm.bind_tools([])
[model for model in ChatNVIDIA.get_available_models() if model.supports_tools]

[Model(id='meta/llama-3.1-405b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='mistralai/mistral-large-2-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.2-3b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-8b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='nv-mistralai/mistral-nemo-12b-instruct', model_type='chat', client='ChatN

In [51]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="NV-Embed-QA", truncate="END")

In [52]:
import os
import requests
import pandas as pd
import urllib.parse  # To handle URL joining
import fitz

from tqdm import tqdm
from io import StringIO
from bs4 import BeautifulSoup, SoupStrainer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, DataFrameLoader, CSVLoader, UnstructuredTSVLoader

class DataHandler:
    """
    Masterfully handles data scraping, preprocessing, and other data-related functionalities in this notebook.
    """

    def __init__(self,
                csv_path="./data/csv"):
        self.visited_urls = set() # classwide tracker to prevent repeated visits
        self.webloaders = [] # tracks all urls that have been converted to langchain WebBaseLoaders.
        self.tabular = [] # tracks all tabular data that has been discovered by scraper. Delivers a list of CSVLoader. Cowardly refusing to save to same list
        self.raws = []
        self.csv_path = csv_path

        self.text_splitter = CharacterTextSplitter(chunk_size=400, separator=" ", chunk_overlap=80)

        self.visited_urls = set()   # Classwide tracker to prevent repeated visits when scraping web
        self.tabular_data = []      # Tracks all tabular_data data that has been discovered by scraper. Delivers a list of CSVLoader. Cowardly refusing to save to same list.
        self.textual_data = []      # Tracks all textual data scraped from websites or pdfs
        self.all_data = []  # Just defining a joint list here instead of later during the functional call; in case we do not care about seperating them anymore (both are lists of Documents, just from different
        # base sources)

        if not os.path.exists(self.csv_path):
            os.mkdir(self.csv_path)
    
    @staticmethod
    def from_csv(csv_path):
        """
        Small func to read from csv and produce CSVLoaders.
        """
        df = pd.from_csv( csv_path) # so we can yoink its columns
        loader = CSVLoader(file_path=csv_path,
                                    csv_args={  'delimiter': ',',
                                                'quotechar': '"',
                                                'fieldnames': [str(col) for col in df.columns]}
                                    )
        return loader

    def csvs_to_loader(self, directory):
        """
        walks through directory, finds all csvs, and saves it into self.tabular 
        """
        for dir, subdir, files in os.walk(directory):
            for file in files:
                if file.endswith(".csv"):
                    fp = os.path.join(dir, file)
                    loader = self.from_csv(fp)
                    self.tabular.append(loader)


    def extract_table_elements(self, url, table_elements):
        """
        Helper func to extract tabular data

        Args:
            - url: url the table is under
            - table_elements: list of table elements
        """
        for table_idx, table in enumerate(table_elements): #TODO: Find better way to index different tables on the same page? not all have class attributes we can ID them with.
            try:
                tags = table.find_all('sup')
                for tag in tags:
                    tag.extract()

                # Attempt to find the title of this table. tableheader elements only tell us (pandas) how to index it, but what we need the header element for context on what this table is about
                tablename = f"table_{table_idx}" # default name presuming none is found
                for headertype in ['h3', 'h4']: # unlikely to lie in h2 or h1? could result in duplicate data. if we find by those.
                    header = table.find_previous(headertype)
                    if header is not None and header.text is not None: # find the closest header
                        tablename = header.text.replace("\n", "").replace("#", '').replace("\r", '').replace("/", '')
                        break
                
                tablename = os.path.basename(url) + " " + f"{tablename}" # what we will call this table, some has really annoying spacing, so maybe .replace(' ', '')?
                print(f"Grabbing table data under tablename {tablename}")
                df = pd.read_html(StringIO(str(table)), header=0)[0] # some tables do not have tableheader <th> tags for first row 
                # which would result in  generic column indices being created, so forcibly set first row as tableheader. 
                df['context'] = [tablename] * df.shape[0]

                if len(tablename) >= os.pathconf('/', 'PC_NAME_MAX'): # prevent shit from exploding because my tablename is damn scuffed but what todo.
                    tablename = tablename[:os.pathconf('/', 'PC_NAME_MAX') - 10]
                csv_path = os.path.join(self.csv_path, tablename + '.csv')
                # print(df)

                # print("csv_path:", csv_path)
                # print("csv_path:", csv_path)
                df.to_csv( csv_path, index=False ) # if true will fuck up columning in csvloader
                # loader = UnstructuredTSVLoader(csv_path, mode='elements')
                loader = CSVLoader(file_path=csv_path,
                                    csv_args={  'delimiter': ',',
                                                'quotechar': '"',
                                                'fieldnames': [str(col) for col in df.columns]}
                                    )
                for row in loader.load()[1:]: # first row is just column indexes, so void
                    self.tabular.append(
                        self.text_splitter.split_text(
                            self.clean_text(row.page_content)
                            )
                        )

            except BaseException as e:
                print(f"Unable to extract table data from url {url} with error {e}, passing!")


    def create_loaders(self):
        """
        Seperate method to create the loaders. Directly appends to textual_data attribute and calls extract_table_elements to handle tabular data.
        """
        for url, soupy_little_guy in self.raws:
            loader = WebBaseLoader(
                    web_paths=(url,),  # No URL fetching as we already have the HTML content
                    bs_kwargs={"parse_only": SoupStrainer(['main'])},
                )
            html_content = loader.load()
            for i in range(len(html_content)):
                self.textual_data.extend(
                    self.text_splitter.split_text(
                        self.clean_text(
                            html_content[i].page_content
                )))

            table_elements = soupy_little_guy.find_all("table") 
            self.extract_table_elements(url=url, table_elements=table_elements)


    def scrape_website(self, base_url, max_depth, depth=0):
        """
        Wraps around a nested function get_from_website, so when ran will define a new function that knows that the sauce base_url is.
        Scuffed? Yes.
        """

        def get_from_website(url, max_depth, depth):
            """
            Recursively scrape a website by visiting links starting from url. 
            Because this is a mostly I/O bound operation, we make a seperate method that actually creates the Loaders.

            Parameters:
                - url:              URL to start scraping from
                - depth:            Current recursion depth
                - max_depth:        Maximum recursion depth to avoid infinite loops

            Returns:
            - appends to self.raws, [url, BeautifulSoup object] created from response content.
            """
            if url in self.visited_urls or depth > max_depth:
                return

            try:
                response = requests.get(url)
                soupy_little_guy = BeautifulSoup(response.content, 'html.parser')
                if response.status_code != 200:
                    return print(f"Failed to retrieve {url}")
            except Exception as e:
                return print(f"Error accessing {url} with error: {e}")
                

            self.visited_urls.add(url)
            print("Current url:", url)
            self.raws.append([url, soupy_little_guy])

            for link in soupy_little_guy.find_all('a', href=True):  # Find all links on the current page
                relative_url = link['href']
                absolute_url = urllib.parse.urljoin(url, relative_url)
                if base_url in absolute_url:  # Avoids external sites
                    get_from_website(absolute_url, max_depth, depth + 1)

        get_from_website(base_url, max_depth, depth)


    def scrape_pdf(self, pdf_path):
        """
        Extracts information from pdf files.
        Texts stay as texts.
        Tables and images...

        Parameters:
            - pdf_path:     PDF file to extract from

        Output:
            - textual_data: List of cleaned strings that represent data from pdf doc
        """
        try:
            pdf_document = fitz.open(pdf_path)
        except:
            return print(f"Unable to open {pdf_path}")
        print(f"Current pdf: {pdf_path}")
        pdf_text = ""
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            pdf_text += page.get_text("text")

        self.textual_data.extend(
            self.text_splitter.split_text(
                self.clean_text(pdf_text)
        ))
    
    def clean_text(self, text):
        """
        Cleans text retrieved from sources to reduce the storage needed

        get_from_website(base_url, max_depth, depth)
        Parameters:
            - text:         Original string

        Output:
            - cleaned_text: Cleaned string 
        """
        return text.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('  ', ' ')
    
    def embed_text(self, text):
        """
        Embeds a given text if necessary

        Parameters:
            - text:     Original string

        Output:
            - text:     Embedded string 
        """
        return embedder.embed_query(text)

In [53]:
# Max depth determines whether you wish to look into children of parents websites, else set to 0
max_depth = 2
websites = [
    # "https://www.iras.gov.sg/taxes",
    "https://www.iras.gov.sg/schemes",
    # "https://www.mom.gov.sg/passes-and-permits",
    # "https://www.mom.gov.sg/employment-practices",
    # "https://www.mom.gov.sg/workplace-safety-and-health"
]

datahandler = DataHandler()
for website in websites:
    datahandler.scrape_website(website, max_depth)

if os.path.isdir('./data/pdfs'):
    for pdf in os.listdir('./data/pdfs'):
        datahandler.scrape_pdf(os.path.join('./data/pdfs', pdf))

Current url: https://www.iras.gov.sg/schemes
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/enterprise-innovation-scheme-(eis)
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/refundable-investment-credit-(ric)
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/jobs-growth-incentive
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/progressive-wage-credit-scheme
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/progressive-wage-credit-scheme/pwcs-glossary
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/uplifting-employment-credit
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/senior-employment-credit-(sec)-cpf-transition-offset-(cto)-and-enabling-employment-credit-(eec)
Current url: https://www.iras.gov.sg/schemes/disbursement-schemes/self-review-for-eligibility-of-government-schemes
Current url: https:

In [54]:
datahandler.create_loaders() # only when we actually want to create loaders to debug stuff, so we don't need to scrape within the same function that we're (likely) debugging, saving time.

Grabbing table data under tablename disbursement-schemes table_0
Grabbing table data under tablename disbursement-schemes table_1
Grabbing table data under tablename disbursement-schemes Details on the Qualifying Conditions for EIS Cash Payout 
Grabbing table data under tablename disbursement-schemes Details on the Qualifying Conditions for EIS Cash Payout 
Grabbing table data under tablename disbursement-schemes             1. Qualifying R&D Undertaken in Singapore        
Grabbing table data under tablename disbursement-schemes 
Grabbing table data under tablename disbursement-schemes 
Grabbing table data under tablename disbursement-schemes 
Grabbing table data under tablename disbursement-schemes 
Grabbing table data under tablename enterprise-innovation-scheme-(eis) table_0
Grabbing table data under tablename enterprise-innovation-scheme-(eis) table_1
Grabbing table data under tablename enterprise-innovation-scheme-(eis) Details on the Qualifying Conditions for EIS Cash Payout 
Gr

In [56]:
# datahandler.embedded_data = [datahandler.embed_text(text) for text in datahandler.textual_data] + [datahandler.embed_text(text) for text in datahandler.tabular_data]
datahandler.all_data = datahandler.textual_data + datahandler.tabular_data

In [57]:
# Here we create a faiss vector store from the documents and save it to disk.
from langchain_community.vectorstores import FAISS

# You will only need to do this once, later on we will restore the already saved vectorstore
store = FAISS.from_texts(datahandler.all_data, embedder)
VECTOR_STORE = './data/nv_embedding'
store.save_local(VECTOR_STORE)

Exception: [402] Payment Required
Account '3z96GEPsEoItReYSMdY7d7W_JmB9XRBYikZWWi4T_rQ': Cloud credits expired - Please contact NVIDIA representatives

In [33]:
from langchain_community.vectorstores import FAISS

# Load the FAISS vectorestore back.
VECTOR_STORE = './data/nv_embedding'
store = FAISS.load_local(VECTOR_STORE, embedder, allow_dangerous_deserialization=True)
retriever = store.as_retriever()

### Creating tools and prompting

Now, we can initialize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see LangChain's [conceptual guide](https://python.langchain.com/v0.1/docs/modules/agents/concepts/)

In [34]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="iras_search",
    description="Search for information under the IRAS organization. \
        For any questions about taxation in Singapore, you must use this tool!",
)

tools = [retriever_tool] # list of possible tools for the agent to use. right now only one.
print(tools)

[Tool(name='iras_search', description='Search for information under the IRAS organization.         For any questions about taxation in Singapore, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f8dbe16cb80>, retriever=VectorStoreRetriever(tags=['FAISS', 'NVIDIAEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8d93355de0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7f8dbe10d480>, retriever=VectorStoreRetriever(tags=['FAISS', 'NVIDIAEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f8d93355de0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables

### NOTE: FOLLOWING DOCUMENTATION MARKDOWN LOOKS VERY MESSY SINCE I SKILL ISSUE. To see just double click and see it in editor mode. 

<!-- Create a template prompt for what the model recieves. The user may input something in, and the agent will that transform that input to be more suited to the model's directives.

Just for understanding to those unfamiliar, here is an explanation of all the classes you see below:

#### PromptTemplate:
- creates a **template** for a prompt; think about this as your f-string pretty much.

prompt_template = PromptTemplate.from_template(
   "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

">>> Tell me a funny joke about chickens."

#### SystemMessagePromptTemplate:
- Exactly what it sounds like, but sent to the AI and not seen by the user.
- Think of this as pre-conditioning the model for expected behaviour.
- Not true guardrailing btw

#### MessagesPlaceHolder:
- Prompt template where the variable is a **list of messages**.
- Acts as a placeholder to pass in a list of messages. See examples for clarity:

prompt = MessagesPlaceholder(variable_name = "history")
prompt.format_messages() ( raises KeyError )

prompt = MessagesPlaceholder(variable_name = "history", optional=True)
prompt.format_messages() ( returns empty list [] )

prompt.format_messages(
history=[
    ("system", "You are an AI assistant."),
    ("human", "Hello!"),
    ]
)

-> [
     SystemMessage(content="You are an AI assistant."),
     HumanMessage(content="Hello!"),
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)

prompt.invoke(
   {
       "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
       "question": "now multiply that by 4"
   }
)

-> ChatPromptValue(messages=[
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="what's 5 + 2"),
    AIMessage(content="5 + 2 is 7"),
    HumanMessage(content="now multiply that by 4"),
]) -->




In [35]:
from langchain_core.prompts.chat import SystemMessagePromptTemplate, \
    PromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, ChatPromptTemplate

templates = [SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant.')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

prompt = ChatPromptTemplate.from_messages(templates)

In [36]:
# makes the agent and agentexecutor

from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
# example questions

# agent_executor.invoke({"input": "hi!"})
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
It's nice to meet you. Is there anything I can assist you with or would you like to chat about something specific?

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `iras_search` with `{'query': 'Testing with Llama/CodeJanitor'}`


please follow the steps below: Launch Configure Java in your computer's Start MenuIn Java Control Panel, under the tab General, click on View... buttonJava Cache Viewer will be launched: Choose Show: ApplicationsSelect S45 Offline Data Entry ApplicationClick to delete the ODE applicationAfter deleting, click Close in the Java Cache ViewerClick OK in Java Control Panel Claiming Double Taxation

selected for review are required to perform the following steps: Self-review of mandatory CPF contributions made for your employeesFor September 2024 PayoutIf you have been selected to conduct a self-review of your eligibility for SEC/EEC/CTO, you would need to check whether there were any errors pertaining to

KeyboardInterrupt: 

### Adding in memory

This agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in previous chat_history. Note: it needs to be called chat_history because of the prompt we are using. If we use a different prompt, we could change the variable name.

In [ ]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

In [ ]:
# another example of langchain agent with memory
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "what's my name?",
    }
)

In [38]:
# automatically keep track of these messages with RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
agent_with_chat_history.invoke(
    {"input": "The current year is 2017, and I am a business looking to claim PIC benefits on my expenditures."},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "PIC"}},
)



> Entering new AgentExecutor chain...
The Productivity and Innovation Credit (PIC) scheme is a taxation incentive provided by the Singapore government. It provides 400% tax relief for donations made to certain registered charities from July 2015 to June 2019

If you are looking to claim PIC benefits on your expenditures, you are likely eligible for the PIC scheme in 2017. Can I help you with anything else?

> Finished chain.


{'input': 'The current year is 2017, and I am a business looking to claim PIC benefits on my expenditures.',
 'chat_history': [],
 'output': 'The Productivity and Innovation Credit (PIC) scheme is a taxation incentive provided by the Singapore government. It provides 400% tax relief for donations made to certain registered charities from July 2015 to June 2019\r\n\r\nIf you are looking to claim PIC benefits on your expenditures, you are likely eligible for the PIC scheme in 2017. Can I help you with anything else?'}

In [39]:
agent_with_chat_history.invoke(
    {"input": "I believe you made a mistake on how the PIC scheme works; it is achieved by \
     6 qualifying activities. Could you correct your statement? "},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "PIC"}},
)



> Entering new AgentExecutor chain...
The Productivity and Innovation Credit (PIC) scheme is a tax incentive provided by the Singapore government for businesses. It achieves 400% tax relief by allowing businesses to claim qualifying expenses incurred on six qualifying activities, which are:

1. Autonomous robots and unmanned systems
2. Computerisation of company processes
3. General data hosting services
4. Quality management software and solutions
5. Software solutions for innovation and productivity
6. Training for innovation and productivity

> Finished chain.


{'input': 'I believe you made a mistake on how the PIC scheme works; it is achieved by      6 qualifying activities. Could you correct your statement? ',
 'chat_history': [HumanMessage(content='The current year is 2017, and I am a business looking to claim PIC benefits on my expenditures.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The Productivity and Innovation Credit (PIC) scheme is a taxation incentive provided by the Singapore government. It provides 400% tax relief for donations made to certain registered charities from July 2015 to June 2019\r\n\r\nIf you are looking to claim PIC benefits on your expenditures, you are likely eligible for the PIC scheme in 2017. Can I help you with anything else?', additional_kwargs={}, response_metadata={})],
 'output': 'The Productivity and Innovation Credit (PIC) scheme is a tax incentive provided by the Singapore government for businesses. It achieves 400% tax relief by allowing businesses to claim qualifying expenses 

In [ ]:
# above answer is completely wrong. Lets try a different prompt:

message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
agent_with_chat_history.invoke(
    {"input": "The current year is 2017, and I am a business looking to claim PIC benefits on my expenditures."},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "PIC"}},
)



> Entering new AgentExecutor chain...
The Productivity and Innovation Credit (PIC) scheme is a taxation incentive provided by the Singapore government. It provides 400% tax relief for donations made to certain registered charities from July 2015 to June 2019

If you are looking to claim PIC benefits on your expenditures, you are likely eligible for the PIC scheme in 2017. Can I help you with anything else?

> Finished chain.


{'input': 'The current year is 2017, and I am a business looking to claim PIC benefits on my expenditures.',
 'chat_history': [],
 'output': 'The Productivity and Innovation Credit (PIC) scheme is a taxation incentive provided by the Singapore government. It provides 400% tax relief for donations made to certain registered charities from July 2015 to June 2019\r\n\r\nIf you are looking to claim PIC benefits on your expenditures, you are likely eligible for the PIC scheme in 2017. Can I help you with anything else?'}